In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 15 20:56:04 2018

@author: corey


Cleaning up the  Yelp Dataset

"""
import pandas as pd
from pprint import pprint
import numpy as np
import json
import censusgeocode as cg

df = pd.read_csv('LA_County_restaurants.csv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
df.head()

df = df[df['city'] == "Los Angeles"]

,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,...,review_count,transactions,url,Latitude,Longitude,city,zipcode,category1,category2,category3
35,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.0047569274902, 'longitude': -1...",(323) 561-1675,54768.500159,fettes-schwein-food-truck-los-angeles-3,https://s3-media3.fl.yelpcdn.com/bphoto/3LOYvP...,False,"{'address1': '', 'address2': '', 'address3': '...",Fettes Schwein Food Truck,1.323561e+10,...,14,[],https://www.yelp.com/biz/fettes-schwein-food-t...,-118.259079,34.004757,Los Angeles,90011,Food Trucks,Hot Dogs,Burgers
45,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.10161, 'longitude': -118.30206}",(323) 477-6764,53442.069549,the-tropic-truck-los-angeles-5,https://s3-media2.fl.yelpcdn.com/bphoto/Fq9EuT...,False,"{'address1': None, 'address2': None, 'address3...",The Tropic Truck,1.323478e+10,...,50,[],https://www.yelp.com/biz/the-tropic-truck-los-...,-118.302060,34.101610,Los Angeles,90027,Food Trucks,Caribbean,Caterers
47,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.1463062227897, 'longitude': -1...",(818) 373-9767,64863.573440,okamoto-kitchen-los-angeles-2,https://s3-media3.fl.yelpcdn.com/bphoto/9bs7YT...,False,"{'address1': None, 'address2': None, 'address3...",Okamoto Kitchen,1.818374e+10,...,168,"['delivery', 'pickup', 'restaurant_reservation']",https://www.yelp.com/biz/okamoto-kitchen-los-a...,-118.425397,34.146306,Los Angeles,91423,Food Trucks,Japanese,Caterers
53,"[{'alias': 'filipino', 'title': 'Filipino'}, {...","{'latitude': 34.0569114685059, 'longitude': -1...",(888) 907-2248,50926.777789,white-rabbit-truck-los-angeles-4,https://s3-media1.fl.yelpcdn.com/bphoto/-_twM9...,False,"{'address1': None, 'address2': None, 'address3...",White Rabbit Truck,1.888907e+10,...,569,[],https://www.yelp.com/biz/white-rabbit-truck-lo...,-118.243950,34.056911,Los Angeles,90012,Filipino,Food Trucks,Caterers
72,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.21332, 'longitude': -118.35861}",(818) 282-4301,58069.947811,shawarma-and-falafel-palace-food-truck-los-ang...,https://s3-media4.fl.yelpcdn.com/bphoto/jKyVhe...,False,"{'address1': '7782 San Fernando Rd', 'address2...",Shawarma and Falafel Palace Food Truck,1.818282e+10,...,1,[],https://www.yelp.com/biz/shawarma-and-falafel-...,-118.358610,34.213320,Los Angeles,91352,Food Trucks,Middle Eastern,NaN
964,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.029109954834, 'longitude': -11...",(562) 794-6484,51357.806229,urban-espresso-los-angeles-6,https://s3-media3.fl.yelpcdn.com/bphoto/bUKrBN...,False,"{'address1': '', 'address2': None, 'address3':...",Urban Espresso,1.562795e+10,...,25,[],https://www.yelp.com/biz/urban-espresso-los-an...,-118.283066,34.029110,Los Angeles,90007,Food Trucks,Coffee & Tea,Breakfast & Brunch
1473,"[{'alias': 'french', 'title': 'French'}]","{'latitude': 34.13904, 'longitude': -118.20416}",(323) 255-5111,12323.773752,cafe-beaujolais-los-angeles,https://s3-media2.fl.yelpcdn.com/bphoto/MZ0MtL...,False,"{'address1': '1712 Colorado Blvd', 'address2':...",Cafe Beaujolais,1.323256e+10,...,1261,[],https://www.yelp.com/biz/cafe-beaujolais-los-a...,-118.204160,34.139040,Los Angeles,90041,French,NaN,NaN
1480,"[{'alias': 'breakfast_brunch', 'title': 'Break...","{'latitude': 34.12768, 'longitude': -118.21722}",(323) 474-6420,14061.604868,rockn-egg-cafe-los-angeles-3,https://s3-media2.fl.yelpcdn.com/bphoto/YwVMqf...,False,"{'address1': '4616 Eagle Rock Blvd', 'address2...",Rock'n Egg Cafe,1.323475e+10,...,33,[],https://www.yelp.com/biz/rockn-egg-cafe-los-an...,-118.217220,34.127680,Los Angeles,90041,Breakfast & Brunch,Diners,Sandwiches
1487,"[{'alias': 'vegan', 'title': 'Vegan'}]","{'latitude': 34.1343840632986, 'longitude': -1...",(323) 775-6614,13367.367100,the-vegan-hooligans-los-angeles,https://s3-media1.fl.yelpcdn.com/bphoto/cXOsCI...,False,"{'address1': '4862 Eagle Rock B

In [100]:
# this variable is for testing extract_location
loc_sample = "{'address1': '349 N State College Blvd', 'address2': None, 'address3': None, 'city': 'Fullerton', 'zip_code': '92831', 'country': 'US', 'state': 'CA', 'display_address': ['349 N State College Blvd', 'Fullerton, CA 92831']}"


def extract_category(df):
    category1 = []
    category2 = []
    category3 = []
    for x in df['categories']:
        json_rec = json.loads(x.replace("'", '"'))
        category1.append(json_rec[0]['title'])
        try:
            category2.append(json_rec[1]['title'])
        except:
            category2.append('')
        try:
            category3.append(json_rec[2]['title'])
        except:
            category3.append('')
    df['category1'] = category1
    df['category2'] = category2
    df['category3'] = category3

extract_category(df)

In [87]:
#pprint(loc_sample)


def extract_location(df):
    '''
    breaks yelp's location data along the key that we want and then
    returns it as a new dataframe
    '''
    cities = []
    zip_codes = []
    streets = []
    #print(df['location'])
    #print(json.dumps(df['location']))
    placeholder = []
    for row in df['location']:
        try:
            address = row.split("'display_address': ['")[1].rstrip("']}").split("', '")
            city = address[1].split(",")[0]
            #print(city)
            cities.append(city)
            street = address[0]
            print(street, city)
            streets.append(address[0])
            #print(address)
        except:
            print('error: \n')
            print(row)
            cities.append('none')
            streets.append('none')
        
        try:
            zip_code = row.split("'zip_code': ")[1].split(",")[0].strip("'")
        except:
            zip_code = np.nan
            
        zip_codes.append(zip_code)
    df['city'] = cities
    df['street'] = streets
    df['zipcode'] = zip_codes
        

       
extract_location(df)    

3165 Yorba Linda Blvd Fullerton
349 N State College Blvd Fullerton
660 N Rose Dr Placentia
2931 Nutwood Ave Fullerton
1325 E Chapman Ave Fullerton
711 N Placentia Ave Fullerton
3203 N Glassell St Orange
114 W Commonwealth Ave Fullerton
305 N State College Blvd Fullerton
2323 E Chapman Ave Ste B
512 N State College Blvd Fullerton
2905 E Miraloma Ave Ste 3
616 W Chapman Ave Placentia
3405 E Imperial Hwy Brea
305 N State College Blvd Fullerton
201 S Bradford Ave Placentia
2279 N Tustin St Orange
1956 N Placentia Ave Fullerton
440 S Anaheim Blvd Ste 104
733 N Placentia Ave Fullerton
1450 N Kraemer Blvd Placentia
1060 E Imperial Hwy Brea
141 E Commonwealth Ave Fullerton
202 W Lincoln Ave Anaheim
2445 E Imperial Hwy Ste M
229 E Commonwealth Ave Fullerton
2555 E Chapman Ave Fullerton
1071 N Tustin Ave Ste 110
2445 E Imperial Hwy Ste F
401 S Magnolia Ave Anaheim
3458 E Orangethorpe Ave Anaheim
1343 E Chapman Ave Fullerton
907 E Yorba Linda Blvd Placentia
374 S Main St Orange
184 S Harbor Blvd 

1313 S Harbor Blvd Anaheim
26676 Portola Pkwy Ste D
2136 Colorado Blvd Los Angeles
440 S Anaheim Blvd No 209-B
10708 Los Alamitos Blvd Los Alamitos
Buena Vista St Anaheim
121 Mc Fadden Pl Newport Beach
13221 Garden Grove Blvd Garden Grove
14258 Newport Ave Tustin
10043 Artesia Pl Bellflower
6740 Greenleaf Ave Whittier
12681 Harbor Blvd Garden Grove
619 N Azusa Ave Azusa
3757 Riverside Dr Ste A
400 W Disney Way Ste 337
46 E Huntington Dr Arcadia
1834 Puente Ave Baldwin Park
7511 Edinger Ave Huntington Beach
228 W 2nd St Pomona
129 E Arrow Hwy Glendora
11332 Washington Blvd Whittier
6339 Etiwanda Ave Mira Loma
5363 E Arrow Hwy Montclair
2114 E Lincoln Ave Anaheim
5555 Hamner Ave Norco
417 S Brookhurst St Anaheim
53 E Union St Pasadena
30 E Orangethorpe Ave Anaheim
10602 Lower Azusa Rd El Monte
1822 Chico Ave South El Monte
2000 E Gene Autry Way Anaheim
11761 Edinger Ave Fountain Valley
820 W 19th St Costa Mesa
16701 Pacific Coast Hwy Sunset Beach
1060 Walnut Ave Tustin
16025 Gale Ave Ind

814 W 19th St Costa Mesa
4523 Gage Ave Bell
3583 N Los Coyotes Diagonal Long Beach
2839 S Diamond Bar Blvd Diamond Bar
1821 W Commonwealth Ave Ste H
194 Marina Dr Ste 101
5621 N Figueroa St Los Angeles
116 E Chapman Ave Orange
4100 Donald Douglas Dr Long Beach
11001 S Central Ave Ontario
9469 Foothill Blvd Rancho Cucamonga
2438 E Katella Ave Anaheim
11265 Topaz Ave Fountain Valley
22722 Ctr Dr Ste A
5100 E 2nd St Long Beach
2493 Park Ave Ste 41
14001 Newport Ave Tustin
1542 W Holt Blvd Ontario
101 Main St Seal Beach
2001 Westcliff Dr Ste 100
error: 

{'address1': '1776 S Disneyland Dr', 'address2': '', 'address3': "Disney's California Adventure", 'city': 'Anaheim', 'zip_code': '92802', 'country': 'US', 'state': 'CA', 'display_address': ['1776 S Disneyland Dr', "Disney's California Adventure", 'Anaheim, CA 92802']}
8228 Firestone Blvd Downey
2142 E 4th St Long Beach
2421 E Artesia Blvd Long Beach
2471 Whittier Blvd Los Angeles
28 W Main St Alhambra
8082 Adams Ave Huntington Beach
4702 W

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11684 entries, 0 to 11683
Data columns (total 22 columns):
categories       11684 non-null object
coordinates      11684 non-null object
display_phone    11338 non-null object
distance         11684 non-null float64
id               11684 non-null object
image_url        11575 non-null object
is_closed        11684 non-null bool
location         11684 non-null object
name             11684 non-null object
phone            11338 non-null float64
price            11192 non-null object
rating           11684 non-null float64
review_count     11684 non-null int64
transactions     11684 non-null object
url              11684 non-null object
Latitude         11684 non-null float64
Longitude        11684 non-null float64
city             11684 non-null object
zipcode          11663 non-null object
category1        11684 non-null object
category2        11684 non-null object
category3        11684 non-null object
dtypes: bool(1), float64(5), in

In [ ]:
import pandas as pd
from pprint import pprint
import numpy as np
import json
import censusgeocode as cg

df = pd.read_csv('LA_County_restaurants.csv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')


results = []
for index,row in df.iterrows():
    lat = round(row['Latitude'],5)
    print(lat)
    lon = round(row['Longitude'],5)
    print(lon)
    
    try:
        response = cg.coordinates(x=lat, y=lon)
        tract = response['2010 Census Blocks'][0]['TRACT']
        print(tract)
        results.append(f"6037{tract}")
    except:
        results.append(np.nan)
        pass
    


-117.6704
34.12313
000815
-117.63614
34.13219
000819
-117.63231
34.1335
000819
-117.76683
34.11298
-117.77612
34.11722
401601
-117.63659
34.13381
000819
-117.75205
34.14777
400205
-117.67154
34.1236
000815
-117.76212
34.11193
401602
-117.77651
34.11736
401601
-117.76287
34.11235
401602
-117.76125
34.11244
401601
-117.81439
34.11767
401311
-117.67092
34.12307
-117.63679
34.13252
-117.6377
34.13377
-117.77384
34.11601
401603
-117.76618
34.11391
401601
-117.76906
34.11443
401602
-117.76172
34.1108
401602
-117.77824
34.11842
-117.79332
34.12221
400302
-117.76541
34.11364
-117.6705
34.12225
000815
-117.76561
34.11315
401602
-117.67004
34.10404
000824
-117.80672
34.12196
-117.70711
34.12096
401800
-117.76607
34.11258
-117.67186
34.12221
-117.76531
34.11383
401601
-117.80022
34.12314
-117.76801
34.11539
401601
-118.31773
33.9011
603500
-117.92455
33.86966
011300
-118.25908
34.00476
-117.70555
34.12149
401800
-117.75964
34.11173
401601
-117.76313
34.11202
401602
-117.76202
34.11279
401601
-117

400304
-117.75711
34.03396
-117.77757
34.0621
402406
-117.81351
33.99781
403323
-117.68542
34.0338
000404
-117.70933
34.0187
000503
-117.71775
34.09514
401902
-117.72108
34.00113
000401
-117.69046
34.06718
000208
-117.80944
34.0198
403319
-117.71704
34.09555
401902
-117.68892
34.03638
000404
-117.71001
34.06373
000301
-117.71301
33.98394
000107
-117.65119
34.03225
001704
-117.68961
34.04011
000404
-117.71965
34.07645
402703
-117.69357
33.96314
-117.6718
34.09083
000825
-117.65353
34.10822
-117.72293
34.01872
000401
-117.72083
34.10727
401800
-117.72147
34.10748
401800
-117.68809
34.03583
000404
-117.74061
34.08135
402102
-117.688
34.00834
000504
-117.7167
34.09515
-117.71303
33.98282
000107
-117.72124
34.10725
401800
-117.67026
34.08903
000825
-117.81439
34.11767
401311
-117.80997
34.00233
403320
-117.7246
34.06255
402702
-117.75931
34.03525
403000
-117.70564
34.01933
000501
-117.81916
34.11755
401311
-117.73717
34.01136
000103
-117.70182
33.98188
000113
-117.74333
34.01324
000103
-117

000826
-117.66889
34.08675
001104
-117.7064
34.03343
000501
-117.71801
34.09486
401902
-117.73534
34.06307
-117.67146
34.12364
000815
-117.81145
34.00233
403319
-117.81636
34.10485
401311
-117.81296
33.9998
403323
-117.67078
34.05624
001600
-117.65021
34.02087
001804
-117.75131
34.10902
401601
-117.70245
33.98231
000113
-117.82001
34.00719
403319
-117.8243
34.10686
401202
-117.68948
34.08146
000205
-117.81385
33.99669
403323
-117.66928
34.10833
-117.66941
34.09332
000826
-117.71388
33.98404
000107
-117.76358
34.11304
401601
-117.73094
34.06271
-117.64578
34.06305
001600
-117.81072
34.00006
403324
-117.73176
34.00308
-117.6309
34.03345
001803
-117.66093
34.06298
-117.70592
33.98223
000113
-117.80995
33.99972
403324
-117.73783
34.08984
402102
-117.81288
33.99977
-117.68984
33.97148
-117.68101
34.01908
000605
-117.66395
34.10773
000818
-117.65563
34.10663
000824
-117.81107
34.12547
400403
-117.64794
34.06296
001400
-117.81216
34.02168
403319
-117.64945
34.06599
001400
-117.70591
34.03295


000826
-117.69083
34.09318
000201
-117.73353
33.98282
000105
-117.7493
34.06285
-117.72231
34.00106
000401
-117.68611
34.1072
000817
-117.71951
34.08025
402001
-117.65151
34.07065
001400
-117.71197
33.98345
000107
-117.70706
34.06415
000301
-117.70657
34.05548
000304
-117.75744
34.03316
403000
-118.28307
34.02911
224700
-117.74936
34.09048
402200
-117.66946
34.09339
000826
-117.65804
34.06376
001101
-117.76925
34.08295
402200
-117.70739
34.0555
000303
-117.63737
34.13307
000819
-117.63671
34.13148
000819
-117.71469
33.99662
000504
-117.85564
33.78783
075902
-117.81828
34.10612
401311
-117.63026
34.10692
000808
-117.77828
34.06249
402406
-117.68217
34.06288
000301
-117.63957
34.13247
000819
-117.74967
34.04761
402803
-117.70171
34.01945
000501
-117.66846
34.03344
001704
-117.65119
34.04005
001704
-117.76663
34.08112
402200
-117.78728
34.06193
402406
-117.6537
34.10622
000824
-117.73585
34.01186
000103
-117.78139
34.07044
402406
-117.70277
34.01853
000503
-117.70741
34.0555
000303
-117.6

900804
-118.1488
34.704
900803
-118.15232
34.60221
910208
-118.14862
34.60192
-118.14838
34.66067
900705
-118.16251
34.85767
005802
-118.04672
34.55945
910711
-118.07552
34.58073
910601
-118.11373
34.67486
900504
-118.09651
34.68834
900506
-118.11373
34.67486
900504
-118.14648
34.70374
900806
-118.09492
34.5805
910502
-118.12894
34.6895
900607
-118.14733
34.67521
900701
-118.09473
34.57954
910502
-118.14827
34.70453
900803
-118.18263
34.66141
901011
-118.141
34.69806
900806
-118.04484
34.5732
910713
-118.13664
34.58302
910401
-118.14082
34.60151
910201
-118.1662
34.67442
901011
-118.0835
34.57943
910602
-118.15259
34.60198
910208
-118.09383
34.68938
900507
-118.1527
34.60408
910208
-118.1523
34.70431
900803
-118.13569
34.68755
-118.15262
34.6745
900705
-118.09508
34.58023
910502
-118.11189
34.70424
900602
-118.14653
34.69705
900806
-118.09195
34.5807
910601
-118.14783
34.66081
900705
-118.08284
34.55879
910603
-118.15077
34.67178
900705
-118.15259
34.60198
-118.14541
34.60218
910201
-1

...

571800
-118.16716
33.77922
576901
-118.07789
33.88596
554700
-118.12171
33.75645
577501
-118.15955
33.8335
571400
-118.10778
33.85978
554521
-118.04717
33.86015
554517
-118.11679
33.89047
553100
-118.13335
33.75982
577400
-118.07277
33.85924
554900
-118.106
33.74021
099511
-118.13233
33.78343
574800
-118.08325
33.88743
-118.1341
33.85253
570800
-118.19322
33.89411
542106
-118.191
33.77002
576200
-118.18484
33.81832
572201
-118.18475
33.82838
571900
-118.15041
33.76823
577100
-118.19248
33.77009
-118.04647
33.79875
110015
-118.13394
33.85043
570800
-118.19082
33.76643
576001
-118.18546
33.76961
576200
-118.14422
33.78273
575002
-118.1031
33.74317
099512
-118.07424
33.83009
-118.12627
33.85977
570800
-118.09561
33.86575
-118.09014
33.85759
554522
-118.14116
33.83256
571200
-118.19225
33.78095
576301
-118.18914
33.77791
576301
-118.12617
33.8967
554001
-118.10541
33.73993
099511
-118.08244
33.87067
554802
-118.07178
33.79879
110015
-118.13167
33.75908
577400
-118.02941
33.80762
110113
-11

554513
-118.17246
33.76771
576602
-118.15147
33.78318
575103
-118.09126
33.8581
554522
-118.1242
33.8838
554204
-118.20858
33.90048
541605
-118.07116
33.83849
555104
-118.08278
33.87425
554801
-118.07199
33.81008
110108
-118.10034
33.74291
-118.1671
33.83279
571504
-118.07107
33.87336
554514
-118.18464
33.8175
572201
-118.05729
33.80286
110014
-118.05523
33.81034
110106
-118.13909
33.78731
575002
-118.10523
33.81067
574000
-118.16794
33.84725
571503
-118.03805
33.85969
110115
-118.07188
33.85949
-118.16053
33.90487
553602
-118.15273
33.88904
-118.1222
33.88944
554101
-118.02974
33.80354
110113
-118.18963
33.77009
576200
-118.13467
33.85373
570800
-118.18666
33.76907
576001
-118.20785
33.7733
980033
-118.0732
33.9202
550100
-118.21803
33.87891
543202
-118.13366
33.75955
577300
-118.15219
33.8607
570100
-118.12557
33.87698
554404
-118.15198
33.88888
553902
-118.08419
33.85717
554900
-118.16581
33.78247
576901
-118.06545
33.83165
555104
-118.15375
33.86033
570100
-118.15105
33.86043
57010

576403
-118.17721
33.84091
571900
-118.15979
33.88936
553901
-118.10598
33.74547
099512
-118.09307
33.85809
554522
-118.07172
33.834
555104
-118.1092
33.75757
-118.07173
33.80973
110108
-118.18909
33.82027
572002
-118.21259
33.82854
544002
-118.18247
33.86022
570601
-118.14159
33.86027
570003
-118.10614
33.81055
574000
-118.19319
33.89376
542106
-118.11349
33.74752
577604
-118.1158
33.8336
-118.08483
33.85832
554900
-118.09808
33.86625
554521
-118.16929
33.91739
-118.1257
33.86828
570002
-118.0726
33.86693
-118.1315
33.75914
-118.13163
33.7592
577400
-118.16324
33.87519
570203
-118.06846
33.8033
110108
-118.15692
33.88939
553901
-118.07213
33.87498
554514
-118.13417
33.86979
554403
-118.02997
33.80317
-118.07488
33.9092
552200
-118.16663
33.78961
-118.17987
33.78248
576401
-118.10146
33.744
099504
-118.12383
33.75668
-118.11929
33.75564
-118.03027
33.91741
503105
-118.15372
33.78998
573403
-118.07123
33.87271
554802
-118.319
33.88707
603600
-118.12554
33.79438
-118.08458
33.85763
-118.

920339
-118.49006
34.42109
920030
-118.5582
34.41719
-118.55132
34.37885
920313
-118.4634
34.41554
920035
-118.4559
34.41591
920037
-118.53052
34.37842
920336
-118.56047
34.4168
920329
-118.55858
34.46273
920108
-118.53195
34.38525
-118.53658
34.42838
920112
-118.46081
34.41613
920035
-118.61956
34.49293
920102
-118.44524
34.32553
-118.56657
34.37693
920326
-118.57159
34.38196
920326
-118.56078
34.4177
920329
-118.5124
34.41683
920013
-118.55958
34.42955
920114
-118.49699
34.44984
920032
-118.62072
34.49253
920102
-118.56749
34.37839
920326
-118.57385
34.3931
920326
-118.56605
34.43793
-118.46298
34.40738
920031
-118.44534
34.4281
920038
-118.57783
34.44287
920107
-118.46227
34.39437
920043
-118.57365
34.39245
920326
-118.56169
34.41793
920329
-118.54052
34.38033
920336
-118.56103
34.41781
920329
-118.62165
34.49574
-118.46555
34.41529
920035
-118.52888
34.38048
920336
-118.57266
34.38884
-118.52734
34.37825
920336
-118.57227
34.388
-118.43499
34.31174
106407
-118.55836
34.41481
920329

113101
-118.58127
34.25735
113101
-118.44848
34.20442
127102
-118.54881
34.1722
139402
-118.58892
34.23326
113421
-118.50204
34.22749
117302
-118.46117
34.15327
141302
-118.53654
34.23775
115202
-118.53655
34.2389
115202
-118.52688
34.22841
115401
-118.61515
34.16817
137201
-118.59646
34.25663
-118.45187
34.22097
-118.47021
34.22156
117408
-118.44878
34.19967
127803
-118.46776
34.22476
117408
-118.60275
34.16926
137502
-118.50137
34.27158
-118.52759
34.20902
-118.50965
34.22025


In [7]:
result = {'Counties': [{'OID': 275901063468976, 'STATE': '06', 'FUNCSTAT': 'A', 'AREAWATER': 1794659470, 'NAME': 'Los Angeles County', 'LSADC': '06', 'CENTLON': '-118.2617650', 'BASENAME': 'Los Angeles', 'INTPTLAT': '+34.1963983', 'COUNTYCC': 'H1', 'MTFCC': 'G4020', 'COUNTY': '037', 'GEOID': '06037', 'CENTLAT': '+34.1957768', 'INTPTLON': '-118.2618616', 'AREALAND': 10510687541, 'COUNTYNS': '00277283', 'OBJECTID': 398, 'CENT': (-118.261765, 34.1957768), 'INTPT': (-118.2618616, 34.1963983)}], 'Census Tracts': [{'OID': 207901115289836, 'STATE': '06', 'FUNCSTAT': 'S', 'NAME': 'Census Tract 1993', 'AREAWATER': 40785, 'LSADC': 'CT', 'CENTLON': '-118.1992140', 'BASENAME': '1993', 'INTPTLAT': '+34.0941911', 'MTFCC': 'G5020', 'COUNTY': '037', 'GEOID': '06037199300', 'CENTLAT': '+34.0926249', 'INTPTLON': '-118.2003961', 'AREALAND': 2550540, 'OBJECTID': 6991, 'TRACT': '199300', 'CENT': (-118.199214, 34.0926249), 'INTPT': (-118.2003961, 34.0941911)}], '2010 Census Blocks': [{'BLKGRP': '1', 'OID': 210404056348637, 'FUNCSTAT': 'S', 'STATE': '06', 'AREAWATER': 0, 'NAME': 'Block 1000', 'SUFFIX': '', 'LSADC': 'BK', 'CENTLON': '-118.1942484', 'LWBLKTYP': 'L', 'BASENAME': '1000', 'BLOCK': '1000', 'INTPTLAT': '+34.1001521', 'MTFCC': 'G5040', 'COUNTY': '037', 'GEOID': '060371993001000', 'CENTLAT': '+34.1001521', 'INTPTLON': '-118.1942484', 'AREALAND': 428518, 'OBJECTID': 3068287, 'TRACT': '199300', 'CENT': (-118.1942484, 34.1001521), 'INTPT': (-118.1942484, 34.1001521)}], 'States': [{'OID': 2749018475066, 'STATE': '06', 'FUNCSTAT': 'A', 'NAME': 'California', 'AREAWATER': 20484627967, 'LSADC': '00', 'CENTLON': '-119.5277460', 'STUSAB': 'CA', 'BASENAME': 'California', 'INTPTLAT': '+37.1551773', 'DIVISION': '9', 'MTFCC': 'G4000', 'STATENS': '01779778', 'GEOID': '06', 'CENTLAT': '+37.1547352', 'INTPTLON': '-119.5434183', 'REGION': '4', 'AREALAND': 403483191859, 'OBJECTID': 14, 'CENT': (-119.527746, 37.1547352), 'INTPT': (-119.5434183, 37.1551773)}]}
# 6037101110 
print(f"6037{result['2010 Census Blocks'][0]['TRACT']}")

6037199300


In [7]:
df['CensusTract'] = results
len(df['CensusTract'])

1895

In [5]:
df.to_csv('LA_County_Resturants_CensusTracts.csv', encoding='utf-8')

In [6]:
len(results)

1930

In [11]:
df.groupby('CensusTract')['price'].value_counts()

CensusTract  price
6037101220   $         1
             $$        1
6037101400   $         1
6037103102   $         1
6037103400   $         1
6037104201   $         2
6037104204   $         1
6037104310   $$        2
6037104320   $         2
6037104401   $         1
6037104500   $         1
6037104610   $         2
6037104620   $$        1
6037104701   $         3
6037104703   $         2
6037104704   $         2
             $$        1
6037104822   $         1
6037106112   $         1
6037106113   $         2
6037106403   $         1
6037106407   $         1
6037106510   $         1
6037107010   $         1
6037108201   $         1
6037109300   $         1
6037109604   $         1
6037111202   $$        1
6037111206   $         1
6037111301   $         3
                      ..
6037553701   $         1
6037600100   $$        1
6037600201   $         1
6037602200   $         1
6037602801   $         1
             $$        1
6037700101   $$        1
6037700300   $         1
      

In [9]:
df.columns

Index(['categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url', 'Latitude',
       'Longitude', 'city', 'zipcode', 'category1', 'category2', 'category3',
       'CensusTract'],
      dtype='object')